# Financial Contracts and Complex Instruments

This notebook demonstrates the implementation and usage of a flexible contract system for financial instruments. The system supports:
- Basic cash flows and payments
- Complex derivative structures
- Callable and autocallable products
- Target redemption notes (TARNs)
- Decision-based optionality

The implementation uses a compositional approach where complex products are built from simpler building blocks.

In [1]:
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
from contract import Contract, Cashflow, Leg, Option
from observable import Observation, Observable, Ticker
from schedule import schedule

## 1. Basic Building Blocks

### 1.1 Contract Hierarchy

The system is built on these core classes:
- `Contract`: Base class for all financial contracts
- `Cashflow`: Represents a single payment with observable amount
- `Leg`: Collection of related cashflows
- `Option`: Represents conditional execution between contracts

Let's start with some basic examples:

In [2]:
# Define some common market observables
libor_6m = Ticker("EUR.LIBOR.6M", "Reuters")
cms_20y = Ticker("EUR.CMS.20Y", "Reuters")
cms_2y = Ticker("EUR.CMS.2Y", "Reuters")

# Create a floating rate payment
fixing_date = datetime(2024, 7, 10)
payment_date = datetime(2024, 7, 12)
notional = 1_000_000

# Simple Libor + spread payment
floating_payment = Cashflow(
    observable=Observation(libor_6m, fixing_date) + 0.002,  # Libor + 20bps
    payment_date=payment_date,
    payment_currency="EUR",
    notional=notional
)

print("Floating rate payment:")
print(floating_payment)

Floating rate payment:
(EUR.LIBOR.6M(2024-07-10) + 0.002)	1000000	EUR	2024-07-12


### 1.2 Structured Payoffs

We can create more complex payoffs using mathematical operations on observables:

In [3]:
# CMS spread option payoff
cms_spread = Cashflow(
    observable=np.max(Observation(cms_20y, fixing_date) - 2 * Observation(cms_2y, fixing_date), 0.0),
    payment_date=payment_date,
    payment_currency="EUR",
    notional=notional
)

print("CMS spread option payoff:")
print(cms_spread)

CMS spread option payoff:
max((EUR.CMS.20Y(2024-07-10) - (EUR.CMS.2Y(2024-07-10) * 2)), 0.0))	1000000	EUR	2024-07-12


## 2. Building Complex Products

### 2.1 Generating Payment Schedules

For products with multiple payments, we first generate the payment schedule:

In [4]:
# Create 10-year schedules for a structured note
start_date = datetime(2024, 7, 10)
tenor = relativedelta(years=10)

# Annual coupon schedule
coupon_schedule = schedule(start_date, tenor, frequency=relativedelta(years=1))
coupon_fixings = [end - relativedelta(days=2) for _, end in coupon_schedule]

# Semi-annual funding schedule
funding_schedule = schedule(start_date, tenor, frequency=relativedelta(months=6))
funding_fixings = [start - relativedelta(days=2) for start, _ in funding_schedule]

print("First 3 coupon dates:")
for start, end in coupon_schedule[:3]:
    print(f"Period: {start.date()} to {end.date()}")

First 3 coupon dates:
Period: 2024-07-10 to 2025-07-10
Period: 2025-07-10 to 2026-07-10
Period: 2026-07-10 to 2027-07-10


### 2.2 Structured Note with Callable Feature

Let's create a callable structured note with:
- Semi-annual funding payments (Libor + spread)
- Annual structured coupons (CMS spread)
- Callable by issuer after first year

In [5]:
# Create funding leg (semi-annual Libor + spread)
funding_leg = Leg([
    Cashflow(
        Observation(libor_6m, fixing) + 0.002,
        payment_date,
        "EUR",
        notional
    ) 
    for fixing, (_, payment_date) in zip(funding_fixings, funding_schedule)
])

# Create coupon leg (annual CMS spread option)
coupon_leg = Leg([
    Cashflow(
        np.max(Observation(cms_20y, fixing) - 2 * Observation(cms_2y, fixing), 0.0),
        payment_date,
        "EUR",
        notional
    )
    for fixing, (_, payment_date) in zip(coupon_fixings, coupon_schedule)
])

# Add callable feature
call_dates = coupon_fixings[1:]  # Callable after first year
call_decisions = [Observation(Ticker("Call", "Issuer"), date) for date in call_dates]

# Build callable structure
from contract import callable
callable_note = callable(
    call_decisions,
    list(coupon_leg.contracts),
    list(funding_leg.contracts),
    cpn_per_call=1,
    funding_per_call=2
)

print("Callable Structured Note:")
print(callable_note)

Callable Structured Note:
(EUR.LIBOR.6M(2024-07-08) + 0.002)	1000000	EUR	2025-01-10
(EUR.LIBOR.6M(2025-01-08) + 0.002)	1000000	EUR	2025-07-10
max((EUR.CMS.20Y(2025-07-08) - (EUR.CMS.2Y(2025-07-08) * 2)), 0.0))	-1000000	EUR	2025-07-10
if Call(2026-07-08) is positive then
    None
else
    (EUR.LIBOR.6M(2025-07-08) + 0.002)	1000000	EUR	2026-01-10
    (EUR.LIBOR.6M(2026-01-08) + 0.002)	1000000	EUR	2026-07-10
    max((EUR.CMS.20Y(2026-07-08) - (EUR.CMS.2Y(2026-07-08) * 2)), 0.0))	-1000000	EUR	2026-07-10
    if Call(2027-07-08) is positive then
        None
    else
        (EUR.LIBOR.6M(2026-07-08) + 0.002)	1000000	EUR	2027-01-10
        (EUR.LIBOR.6M(2027-01-08) + 0.002)	1000000	EUR	2027-07-10
        max((EUR.CMS.20Y(2027-07-08) - (EUR.CMS.2Y(2027-07-08) * 2)), 0.0))	-1000000	EUR	2027-07-10
        if Call(2028-07-08) is positive then
            None
        else
            (EUR.LIBOR.6M(2027-07-08) + 0.002)	1000000	EUR	2028-01-10
            (EUR.LIBOR.6M(2028-01-08) + 0.002)	1000000	

### 2.3 Target Redemption Note (TARN)

A TARN automatically redeems when cumulative coupons reach a target level. Let's modify our structure to include this feature:

In [6]:
# Create cumulative coupon observations
coupons = [np.max(Observation(cms_20y, fixing) - 2 * Observation(cms_2y, fixing), 0.0) 
           for fixing in coupon_fixings]
cumulative_coupons = np.cumsum(coupons)

# TARN with 10% target
target_level = 0.10  # 10%
tarn = callable(
    cumulative_coupons - target_level,  # Triggers when cumulative coupons exceed target
    list(coupon_leg.contracts),
    list(funding_leg.contracts),
    cpn_per_call=1,
    funding_per_call=2
)

print("TARN Structure:")
print(tarn)

TARN Structure:


if (max((EUR.CMS.20Y(2025-07-08) - (EUR.CMS.2Y(2025-07-08) * 2)), 0.0)) - 0.1) is positive then
    None
else
    (EUR.LIBOR.6M(2024-07-08) + 0.002)	1000000	EUR	2025-01-10
    (EUR.LIBOR.6M(2025-01-08) + 0.002)	1000000	EUR	2025-07-10
    max((EUR.CMS.20Y(2025-07-08) - (EUR.CMS.2Y(2025-07-08) * 2)), 0.0))	-1000000	EUR	2025-07-10
    if ((max((EUR.CMS.20Y(2025-07-08) - (EUR.CMS.2Y(2025-07-08) * 2)), 0.0)) + max((EUR.CMS.20Y(2026-07-08) - (EUR.CMS.2Y(2026-07-08) * 2)), 0.0))) - 0.1) is positive then
        None
    else
        (EUR.LIBOR.6M(2025-07-08) + 0.002)	1000000	EUR	2026-01-10
        (EUR.LIBOR.6M(2026-01-08) + 0.002)	1000000	EUR	2026-07-10
        max((EUR.CMS.20Y(2026-07-08) - (EUR.CMS.2Y(2026-07-08) * 2)), 0.0))	-1000000	EUR	2026-07-10
        if (((max((EUR.CMS.20Y(2025-07-08) - (EUR.CMS.2Y(2025-07-08) * 2)), 0.0)) + max((EUR.CMS.20Y(2026-07-08) - (EUR.CMS.2Y(2026-07-08) * 2)), 0.0))) + max((EUR.CMS.20Y(2027-07-08) - (EUR.CMS.2Y(2027-07-08) * 2)), 0.0))) - 

## 3. Advanced Features

### 3.1 Decision-Based Optionality

The system supports explicit modeling of decisions, which is useful for:
- Settlement calculations
- Bermudan option pricing using Longstaff-Schwartz
- Smoothing digital payoffs for path-wise AAD

In [7]:
# Example: European option with explicit exercise decision
strike = 100.0
expiry = datetime(2024, 7, 10)
stock = Ticker("AAPL", "Yahoo")

# Create option with counterparty decision
exercise_decision = Observation(Ticker("TradeID_exercise", "Counterparty"), expiry)
option_payoff = Option(
    condition=exercise_decision,
    contract1=Cashflow(Observation(stock, expiry), expiry, "EUR", notional) - 
             Cashflow(strike, expiry, "EUR", notional),
    contract2=None
)

print("Option with explicit exercise decision:")
print(option_payoff)

Option with explicit exercise decision:
if TradeID_exercise(2024-07-10) is positive then
    AAPL(2024-07-10)	1000000	EUR	2024-07-10
    100.0	-1000000	EUR	2024-07-10
else
    None
